In [1]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import keras
import os
import numpy as np
from tensorflow.image import resize_with_pad, grayscale_to_rgb
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout, Conv2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential

PATH = "/mnt/d/Datasets/mini-DDSM/"
imgPATH = PATH + "MINI-DDSM-Complete-PNG-16/"
roi_path = PATH + "ROI/"
save_dir = "/home/jj/FYP/Pass_test/"
image_size = 224
batch_size = 8

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

os.environ['TF_ENABLE_GPU_GARBAGE_COLLECTION'] = 'false'

2023-01-17 00:15:01.454536: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-17 00:15:02.429976: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jj/miniconda3/envs/wsl_tf/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda/include:/usr/local/cuda/lib64:/usr/local/cuda-11.2/lib64::/usr/local/cuda/extras/CUPTI/lib64
2023-01-17 00:15:02.430080: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared ob

In [2]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  roi_path,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(image_size, image_size),
  batch_size=batch_size)
  
val_ds = tf.keras.utils.image_dataset_from_directory(
  roi_path,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(image_size, image_size),
  batch_size=batch_size)


Found 2830 files belonging to 2 classes.
Using 2264 files for training.


2023-01-17 00:15:07.606399: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-17 00:15:07.607697: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-17 00:15:07.607740: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-17 00:15:07.607755: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_nod

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
def prepare(ds, shuffle=False, augment=False):
  # # Resize and rescale all datasets.
  
  data_augmentation = tf.keras.Sequential([
  # Input(shape=(None,None,3)),
  tf.keras.layers.RandomFlip('horizontal_and_vertical'),
  tf.keras.layers.RandomRotation((0,0.3),fill_mode="constant"),
  # tf.keras.layers.RandomZoom(height_factor=(-0.5,0.5),width_factor=(-0.5,0.5),fill_mode="constant"),
  # tf.keras.layers.RandomTranslation(height_factor=(-0.25,0.25),width_factor=(-0.25,0.25),fill_mode="constant")
  ])
  if shuffle:
    ds = ds.shuffle(1000)

  # Batch all datasets.
  # ds = ds.batch(batch_size)

  # Use data augmentation only on the training set.
  if augment:
    # ds = ds.map(lambda x, y: (resize_and_rescale(x), y), 
    #           num_parallel_calls=AUTOTUNE)
    ds = ds.map(lambda x, y: (data_augmentation(x,training=True),y), 
                num_parallel_calls=AUTOTUNE)

  # Use buffered prefetching on all datasets.
  return ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
train_ds_aug = prepare(train_ds, shuffle=True, augment=False)
val_ds = prepare(val_ds)

# Training dense layers to categorise output from CNN models

In [ ]:
# preprocessing = tf.keras.applications.inception_v3.preprocess_input
# pre_trained = tf.keras.applications.inception_v3.InceptionV3(weights='imagenet', include_top=False, input_shape=(image_size,image_size,3))
# pre_trained.trainable = False
# #Define model architect
# tfinput = Input(shape=(image_size,image_size,3) )
# pre_process = preprocessing(tfinput)
# inceptionv3_model=pre_trained(pre_process,training=False)
# flatten = GlobalAveragePooling2D()(inceptionv3_model)
# DO1 = Dropout(0.5)(flatten)
# Dense1 = Dense(256,activation = 'relu')(DO1)
# DO2 = Dropout(0.8)(Dense1)
# # DO2 = Dropout(0.8)(flatten)
# output = Dense(1, activation="sigmoid")(DO2)
# model = Model(tfinput,output)
# #Prevent overfitting with early stopping
# Earlystop = tf.keras.callbacks.EarlyStopping( monitor='val_accuracy'
#                                               ,patience=10
#                                               # ,start_from_epoch = 0
#                                               # ,baseline = 0.7
#                                               ,restore_best_weights = True
#                                               ,verbose = 1
#                                               )
# model.compile(
#     optimizer= tf.keras.optimizers.Adam(learning_rate = 1e-4),
#     loss=tf.keras.losses.BinaryCrossentropy(),
#     metrics=["accuracy"]
# )
# model.summary()

In [ ]:
# #Training Dense layers to initialise the categoriser before fine tuning the CNN models
# total_epochs = 50
# history_init = model.fit(train_ds_aug
#                     ,epochs=total_epochs
#                     ,validation_data=val_ds
#                     ,callbacks = Earlystop
#                     )
# # model.save(save_dir + "inceptionv3")
# plt.plot(history_init.history['loss'])

Train and save categoriser(Dense layers)

Load into model to continue training

In [ ]:
# inceptionv3_categoriser_input = model.get_layer(name = "dropout").get_output_at(0)
# inceptionv3_categoriser_output = model.get_layer(name = "dense_1").get_output_at(0)
# inceptionv3_categoriser = Model(inceptionv3_categoriser_input, inceptionv3_categoriser_output)
# # inceptionv3_categoriser.summary()
# inceptionv3_categoriser.save("/home/jj/FYP/Pass_test/incpetionv3_categoriser")

# Load categoriser into model for fine tuning

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal_and_vertical'),
  tf.keras.layers.RandomRotation((0,0.3),fill_mode="constant"),
  tf.keras.layers.RandomZoom(height_factor=(-0.5,0.5),width_factor=(-0.5,0.5),fill_mode="constant"),
  tf.keras.layers.RandomTranslation(height_factor=(-0.25,0.25),width_factor=(-0.25,0.25),fill_mode="constant")
  ])
preprocessing = tf.keras.applications.inception_v3.preprocess_input
pre_trained = tf.keras.applications.inception_v3.InceptionV3(weights='imagenet', include_top=False, input_shape=(image_size,image_size,3))
pre_trained.trainable = True
for layer in pre_trained.layers[:-64]:
    layer.trainable = False
tfinput = Input(shape=(image_size,image_size,3) )
data_augment = data_augmentation(tfinput)
pre_process = preprocessing(data_augment)
inceptionv3_model=pre_trained(pre_process,training=False)
categoriser = load_model("/home/jj/FYP/Pass_test/incpetionv3_categoriser")
pooling = GlobalAveragePooling2D()(inceptionv3_model)
output = categoriser(pooling,training=True)
model = Model(tfinput,output)
model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate = 1e-6),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"]
)
# Earlystop = tf.keras.callbacks.EarlyStopping( monitor='val_accuracy'
#                                               ,patience=10
#                                               # ,start_from_epoch = 20
#                                               # ,baseline = 0.7
#                                               ,restore_best_weights = True
#                                               ,verbose = 1
#                                               )
model.summary()

Checkpoint = tf.keras.callbacks.ModelCheckpoint(  
                                                  filepath="/home/jj/FYP/Pass_test/inceptionv3_best_model"
                                                , save_weights_only=True
                                                , monitor='val_loss'
                                                , mode = 'min'
                                                , save_best_only=True
                                                , verbose=1
                                                )

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential_4 (Sequential)   (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv_2 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_2 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d_2   (None, 2048)             0   

In [ ]:
total_epochs = 100
history_inceptionv3_init = model.fit(train_ds_aug
                    ,epochs=total_epochs
                    ,validation_data=val_ds
                    ,callbacks = [Checkpoint]
                    )
# if(history_inceptionv3_init.history['accuracy'])
model.save(save_dir + "Inceptionv3_TL_FT")

Epoch 1/100
283/283 [==============================] - ETA: 0s - loss: 0.6700 - accuracy: 0.5760

AttributeError: 'NoneType' object has no attribute 'get'

In [18]:
x=0
while history_inceptionv3_init.history['accuracy'][-1]<0.8 and x<5:
    start_epoch = history_inceptionv3_init.epoch[-1] + 1
    total_epochs = start_epoch+50
    history_inceptionv3_init = model.fit(train_ds_aug
                    ,epochs=total_epochs
                    ,validation_data=val_ds
                    ,callbacks = Earlystop
                    ,initial_epoch=start_epoch
                    )
    if history_inceptionv3_init.history['accuracy'][-1] < 0.6:
        x=x
    else:
        x=x+1
    

Epoch 17/66
283/283 [==============================] - 16s 48ms/step - loss: 0.6159 - accuracy: 0.6533 - val_loss: 0.6159 - val_accuracy: 0.6678
Epoch 18/66
283/283 [==============================] - 16s 48ms/step - loss: 0.6161 - accuracy: 0.6568 - val_loss: 0.6213 - val_accuracy: 0.6802
Epoch 19/66
283/283 [==============================] - 16s 48ms/step - loss: 0.6048 - accuracy: 0.6696 - val_loss: 0.6392 - val_accuracy: 0.6608
Epoch 20/66
283/283 [==============================] - 16s 48ms/step - loss: 0.6080 - accuracy: 0.6599 - val_loss: 0.6215 - val_accuracy: 0.6820
Epoch 21/66
283/283 [==============================] - 16s 48ms/step - loss: 0.5997 - accuracy: 0.6789 - val_loss: 0.6221 - val_accuracy: 0.6784
Epoch 22/66
283/283 [==============================] - 16s 48ms/step - loss: 0.5981 - accuracy: 0.6625 - val_loss: 0.6353 - val_accuracy: 0.6731
Epoch 23/66
283/283 [==============================] - 16s 48ms/step - loss: 0.5954 - accuracy: 0.6670 - val_loss: 0.6113 - val_ac

: 

: 

# Others


In [ ]:
# Freeze all the layers except the last 2 layers
tf.keras.backend.clear_session()
pre_trained.trainable = True
for layer in pre_trained.layers[:-64]:
    layer.trainable = False
# pre_trained.summary()
tfinput = Input(shape=(image_size,image_size,3) )
data_augment = data_augmentation(tfinput)
pre_process = preprocessing(data_augment)
# pre_process = preprocessing(tfinput)
inceptionv3_model=pre_trained(pre_process,training=False)
flatten = GlobalAveragePooling2D()(inceptionv3_model)
# flatten = Flatten()(inceptionv3_model)
# Dense1 = Dense(255, activation="relu")(flatten)
# DO1 = Dropout(0.8)(Dense1)
# Dense2 = Dense(127, activation="relu")(DO1)
# DO2 = Dropout(0.8)(Dense2)

DO1 = Dropout(0.8)(flatten)
Dense1 = Dense(127,activation = 'relu')(DO1)
# DO2 = Dropout(1)(Dense1)

DO2 = Dropout(0.8)(Dense1)
output = Dense(1, activation="sigmoid")(DO2)
model = Model(tfinput,output)
model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate = 1e-5),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"]
)
model.summary()

In [ ]:
# Earlystop = tf.keras.callbacks.EarlyStopping( monitor='val_accuracy',
#                                               patience=20,
#                                               start_from_epoch = 0,
#                                               # baseline = 0.7,
#                                               # restore_best_weights = True,
#                                               verbose = 1
#                                               )
total_epochs = history_inceptionv3_init.epoch[-1]+1+ 30
history_inceptionv3_ft = model.fit(train_ds
                    ,epochs=total_epochs
                    ,validation_data=val_ds
                    ,initial_epoch=history_inceptionv3_init.epoch[-1]+1
                    # ,callbacks = Earlystop
                    )
model.save(save_dir + "Inceptionv3_TL_FT")


###Other Tests


In [ ]:
tf.keras.backend.clear_session()
preprocessing = tf.keras.applications.inception_v3.preprocess_input
pre_trained = tf.keras.applications.inception_v3.InceptionV3(weights='imagenet', include_top=False, input_shape=(image_size,image_size,3))
# Freeze all the layers except the last 2 layers
for layer in pre_trained.layers[:-64]:
    layer.trainable = False
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal_and_vertical'),
  tf.keras.layers.RandomRotation((0,0.3),fill_mode="constant"),
  tf.keras.layers.RandomZoom(height_factor=(-0.5,0.5),width_factor=(-0.5,0.5),fill_mode="constant"),
  tf.keras.layers.RandomTranslation(height_factor=(-0.25,0.25),width_factor=(-0.25,0.25),fill_mode="constant")
  ])

tfinput = Input(shape=(image_size,image_size,3) )
data_augment = data_augmentation(tfinput)
pre_process = preprocessing(data_augment)
inceptionv3_model=pre_trained(pre_process,training=True)
flatten = Flatten()(inceptionv3_model)
Dense1 = Dense(255, activation="relu")(flatten)
DO1 = Dropout(0.8)(Dense1)
Dense2 = Dense(127, activation="relu")(DO1)
DO2 = Dropout(0.8)(Dense2)
output = Dense(1, activation="sigmoid")(DO2)
model = Model(tfinput,output)

model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate = 1e-5),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"]
)
model.summary()
total_epochs = 100
history = model.fit(train_ds_aug,
                    epochs=total_epochs,
                    validation_data=val_ds,
                    callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
                    )

In [ ]:
tfinput = Input(shape=(image_size,image_size,3) )
print(tfinput)
# x = data_augmentation(tfinput)
x = preprocessing(tfinput)
# print(x)
x=pre_trained(x,training=True)
# print(x)
x = Flatten()(x)
print(x)
x = Dense(255, activation="relu")(x)
x = Dropout(0.8)(x)
x = Dense(127, activation="relu")(x)
x = Dropout(0.8)(x)
output = Dense(1, activation="sigmoid")(x)
model = Model(tfinput,output)

model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate = 1e-6),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"]
)
model.summary()

In [ ]:
history = model.fit(train_ds
                    ,epochs=10
                    ,validation_data=val_ds
                    # ,callbacks = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
                    )

In [ ]:
fine_tune_epochs = 10
total_epochs =  history.epoch[-1] + fine_tune_epochs

model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate = 1e-5),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"]
)

history_fine = model.fit(train_ds,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=val_ds)

Accuracy = history_fine.history['val_loss']
plt.plot(Accuracy)
plt.show()

In [ ]:
fine_tune_epochs = 100
total_epochs =  history.epoch[-1] + fine_tune_epochs

model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate = 1e-5),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"]
)

history_fine = model.fit(train_ds,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=val_ds)

Accuracy = history_fine.history['val_accuracy']
plt.plot(Accuracy)
plt.show()

In [ ]:
model.save("/home/jj/FYP/Pass_test/mini-DDSM_Inceptionv3_model")

In [ ]:
tf.keras.backend.clear_session()
preprocessing = tf.keras.applications.vgg16.preprocess_input
pre_trained = VGG16(weights='imagenet', include_top=False, input_shape=(image_size,image_size,3))
# Freeze all the layers except the last 2 layers
for layer in pre_trained.layers[:-12]:
    layer.trainable = False
tfinput = Input(shape=(image_size,image_size,3) )
print(tfinput)
# x = data_augmentation(tfinput)
x = preprocessing(tfinput)
# print(x)
x=pre_trained(x,training=True)
# print(x)
x = Flatten()(x)
print(x)
x = Dense(255, activation="relu")(x)
x = Dropout(0.8)(x)
x = Dense(127, activation="relu")(x)
x = Dropout(0.8)(x)
output = Dense(1, activation="sigmoid")(x)
model = Model(tfinput,output)

model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate = 1e-6),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"]
)
model.summary()
epochs_per_cycle = 10
init_epoch = 1
total_epochs = 10
for i in range(10):
    train_ds_aug = prepare(train_ds, shuffle=True, augment=True)
    history = model.fit(train_ds,
                         epochs=total_epochs,
                         initial_epoch=init_epoch,
                         validation_data=val_ds)
    init_epoch = init_epoch + epochs_per_cycle
    total_epochs = total_epochs + epochs_per_cycle

model.save("/home/jj/FYP/Pass_test/mini-DDSM_VGG16_model")

In [ ]:
tf.keras.backend.clear_session()
preprocessing = tf.keras.applications.resnet50.preprocess_input
pre_trained = tf.keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(image_size,image_size,3))
# Freeze all the layers except the last 2 layers
for layer in pre_trained.layers[:-22]:
    layer.trainable = False
tfinput = Input(shape=(image_size,image_size,3) )
print(tfinput)
# x = data_augmentation(tfinput)
x = preprocessing(tfinput)
# print(x)
x=pre_trained(x,training=True)
# print(x)
x = Flatten()(x)
print(x)
x = Dense(255, activation="relu")(x)
x = Dropout(0.8)(x)
x = Dense(127, activation="relu")(x)
x = Dropout(0.8)(x)
output = Dense(1, activation="sigmoid")(x)
model = Model(tfinput,output)

model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate = 1e-5),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"]
)
model.summary()
epochs_per_cycle = 10
init_epoch = 0
total_epochs = 10
for i in range(10):
    train_ds_aug = prepare(train_ds, shuffle=True, augment=True)
    history = model.fit(train_ds,
                         epochs=total_epochs,
                         initial_epoch=init_epoch,
                         validation_data=val_ds)
    init_epoch = init_epoch + epochs_per_cycle
    total_epochs = total_epochs + epochs_per_cycle

model.save("/home/jj/FYP/Pass_test/mini-DDSM_resnet50_model")

In [ ]:
tf.keras.backend.clear_session()
preprocessing = tf.keras.applications.inception_v3.preprocess_input
pre_trained = tf.keras.applications.inception_v3.InceptionV3(weights='imagenet', include_top=False, input_shape=(image_size,image_size,3))
# Freeze all the layers except the last 2 layers
for layer in pre_trained.layers[:-64]:
    layer.trainable = False
tfinput = Input(shape=(image_size,image_size,3) )
print(tfinput)
# x = data_augmentation(tfinput)
x = preprocessing(tfinput)
# print(x)
x=pre_trained(x,training=True)
# print(x)
x = Flatten()(x)
print(x)
x = Dense(255, activation="relu")(x)
x = Dropout(0.8)(x)
x = Dense(127, activation="relu")(x)
x = Dropout(0.8)(x)
output = Dense(1, activation="sigmoid")(x)
model = Model(tfinput,output)

model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate = 1e-5),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"]
)
model.summary()
epochs_per_cycle = 10
init_epoch = 0
total_epochs = 10
for i in range(10):
    train_ds_aug = prepare(train_ds, shuffle=True, augment=True)
    history = model.fit(train_ds,
                         epochs=total_epochs,
                         initial_epoch=init_epoch,
                         validation_data=val_ds)
    init_epoch = init_epoch + epochs_per_cycle
    total_epochs = total_epochs + epochs_per_cycle

model.save("/home/jj/FYP/Pass_test/mini-DDSM_inceptionv3_model")

In [ ]:
tf.keras.backend.clear_session()
preprocessing = tf.keras.applications.efficientnet_v2.preprocess_input
pre_trained = tf.keras.applications.efficientnet_v2.EfficientNetV2S(weights='imagenet', include_top=False, input_shape=(image_size,image_size,3))
# Freeze all the layers except the last 2 layers
for layer in pre_trained.layers[:-75]:
    layer.trainable = False
tfinput = Input(shape=(image_size,image_size,3) )
print(tfinput)
# x = data_augmentation(tfinput)
x = preprocessing(tfinput)
# print(x)
x=pre_trained(x,training=True)
# print(x)
x = Flatten()(x)
print(x)
x = Dense(255, activation="relu")(x)
x = Dropout(0.8)(x)
x = Dense(127, activation="relu")(x)
x = Dropout(0.8)(x)
output = Dense(1, activation="sigmoid")(x)
model = Model(tfinput,output)

model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate = 1e-5),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"]
)
model.summary()
epochs_per_cycle = 10
init_epoch = 0
total_epochs = 10
for i in range(10):
    train_ds_aug = prepare(train_ds, shuffle=True, augment=True)
    history = model.fit(train_ds,
                         epochs=total_epochs,
                         initial_epoch=init_epoch,
                         validation_data=val_ds)
    init_epoch = init_epoch + epochs_per_cycle
    total_epochs = total_epochs + epochs_per_cycle

model.save("/home/jj/FYP/Pass_test/mini-DDSM_efficientnet_v2_model")

In [ ]:
pre_trained = tf.keras.applications.efficientnet_v2.EfficientNetV2S(weights='imagenet', include_top=False, input_shape=(image_size,image_size,3))